Conda environment: dbstepenv

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops
import dbstep.Dbstep as db
import itertools, os
import pandas as pd

# 1) Get path, XYZ, DBStep

In [ ]:
def smiles2xyz(smiles:str) ->str:
    """Take a SMILES string, generate conformers, and produce a temporary XYZ file"""
    ### Create molecule and add hydrogens
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    ### Embed molecule (generates random 3D coordinates)
    AllChem.EmbedMolecule(mol)
    ### Optimize structure of molecule
    AllChem.MMFFOptimizeMolecule(mol)
    ### Set up conformational search program
    params = AllChem.ETKDGv3()
    ### Generate 10 conformers using ETKDGv3 method
    cids = AllChem.EmbedMultipleConfs(mol,10,params)
    ### Optimize structure of conformers
    res = AllChem.MMFFOptimizeMoleculeConfs(mol)
    ### Identify lowest energy conformation
    sortedres = [[n,x[1]] for n,x in enumerate(res)]
    sortedres.sort(key=lambda x: x[1])
    lowestmol = Chem.Mol(mol, confId=sortedres[0][0])
    ### Write RDKit mol to temporary XYZ file
    xyzfile = 'tempfiles/TEMPxyz.xyz'
    Chem.MolToXYZFile(mol,xyzfile)
    return xyzfile

def getlongestpath(mol:rdkit.Chem.rdchem.Mol)-> list:
    """Given a RDKit mol object, find the longest path between atoms"""
    if len(mol.GetAtoms()) ==1: ### If only one atom present (ex: H), return empty list
        return [[0,0,0]] ### path length: 0, start atom: 0, end atom: 0
    ### Get all combinations of atom pairs in the molecule
    atomcombos = itertools.permutations(list(range(len(mol.GetAtoms()))), 2)
    ### Find the length of the shortest (most direct) path between the atom pair, starting atom index, ending atom index
    atomcombos = [(len(rdmolops.GetShortestPath(mol,x[0],x[1])),x[0],x[1]) for x in atomcombos]
    ### Find the longest length
    maxcombo = max([y[0] for y in atomcombos])
    ### Get atom pairs whose length equals the max length (possible for multiple paths to be valid)
    atomcombos = [x for x in atomcombos if x[0]==maxcombo]
    return atomcombos

def getdbstepresults(xyzfile:str,path:str)->dict:
    """Taking XYZ file and atom pair (path) as input, calculate sterimol and volume parameters"""
    mol = db.dbstep(xyzfile,atom1=path[0],atom2=path[1],commandline=True,verbose=False,sterimol=True,measure='classic')
    vol = db.dbstep(xyzfile,atom1=path[0],commandline=True,verbose=False,sterimol=False,r=3.5,b=True,measure='classic')
    return {'atom1':path[0],'atom2':path[1],'L':mol.L,'Bmin':mol.Bmin,'Bmax':mol.Bmax,'PercentBuried_Volume':vol.bur_vol,'Percent_Buried_Sheel':vol.bur_shell,'Occupied_Volume':vol.occ_vol}    

def smileslist2dbstep(smileslist:list)->dict:
    """Taking a list of SMILES, create temporary XYZ files, calculate longest path, and calculate sterimol parameters."""
    results = {}
    for s in smileslist:
        mol = Chem.MolFromSmiles(s)
        longestpath = getlongestpath(mol)
        xyzfile = smiles2xyz(s)
        for n,p in enumerate(longestpath): 
            ## Generate multiple possibilities if there are multiple equally long routes
            results[f'{s}_{n}'] = getdbstepresults(xyzfile,p)
    return results

# smilesls = ['[H][H]','C','C=CC=C','C1=CC=C1','c1ccccc1','OC','C(=O)OC','CC','CCCC','C1CC=CCC1','C(C)(C)','SC','C#N','O','COC1=CC(OC)=CC(OC)=C1']
smilesls = [ 'O=C1OCC[N]1',
 'ClCl',
 'Cl',
 'Br','F','BrBr','FF',
 'BrC1=CC=CC=C1',
 'C',
 'C#C',
 'C#N',
 'C(=O)OC',
 'C(C)(C)',
 'C1(C=CC=C2)=C2SC=N1',
 'C1(CC2=CC=CC=C2)=CC=CC=C1',
 'C1=CC=C1',
 'C1=CC=CC=N1',
 'C1=CC=CN=C1',
 'C1=CC=CO1',
 'C1=CC=CS1',
 'C1=CC=NC=C1',
 'C1=CC=NC=N1',
 'C1=CCCC1',
 'C1=CNC=C1',
 'C1=COC=C1',
 'C1CC=CCC1',
 'C1CCC1',
 'C1CCC=CO1',
 'C=C',
 'C=CC=C',
 'CC',
 'CC(C)C',
 'CCCC',
 'COC1=CC(OC)=CC(OC)=C1',
 'COC1=CC(OC)=CC=C1',
 'C[Si](C)C',
 'FC(C1=CC=CC=C1)(F)F',
 'NC=O',
 'O',
 'O=C1OCNC1',
 'O=[P](OC)OC',
 'OC',
 'SC',
 '[H][H]',
 '[NH2]',
 '[NH]C',
 'c1ccccc1',
 'C(F)(F)(F)',
 'C(F)(F)(F)(F)'
]

### Create a directory for the temporary XYZ files if neccesary
# os.mkdir('tempfiles')

### Generate dataframe, invert XY axes, rename index, eliminate any paramaters with no width (Bmin)
df = pd.DataFrame(smileslist2dbstep(smilesls)).T.reset_index().rename(mapper={'index':'smiles'},axis=1)
df = df[df['Bmin']!=0]
df.to_csv('DBStep_Parameters_GitHub.csv',index=False)
df